# Leaked GiveSendGo data

In [2]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [45]:
import pandas as pd
import geopandas as gpd
import json
import altair as alt

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

## Read data

In [8]:
src = pd.read_excel(
    "data/raw/convoy_donations_leaked.xlsx",
    dtype={"service_provider_credentials_id": str, "donation_zip": str},
)

In [231]:
src.head()

,donation_id,campaign_id,donation_user_id,service_provider_credentials_id,donation_status_id,donation_type,donation_anonymous,donation_date,donation_name,donation_first_name,donation_last_name,donation_email,donation_zip,donation_country,donation_ip,donation_comment,donation_amount,to_gsg_amount,donation_service_fee,donation_service_provider_fee,donation_net_amount,donation_service_provider_transaction,donation_fee_id,stripe_subscriptions_id,donation_charge_id,donation_gsg_charge_id,donation_amount_refunded,donation_stripe_withheld,donation_gsg_charge_failed,donation_conversion_rate,to_gsg_amount_reporting,donation_stripe_webhook_json,donation_recurring_id,brand,funding,campaign_percent_charge,donation_comment_reply,reference_id,retired,employer,occupation,giver_phone,referral_code,referral_member_id
0,463549,49000,NaN,22797,1,1,0,2022-02-01 20:30:18,Jeff Brain,Jeffrey,Brain,jeff@Clouthub.com,92679,US,172.31.27.134,We support the Truckers standing up for our freedom.,100.0,10.0,0,0.0,0,NaN,NaN,NaN,ch_3KOSzTH7qsxOFoUf0U8qVDS0,NaN,NaN,0,0,1,0,NaN,NaN,MasterCard,debit,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,464203,49000,NaN,22797,1,1,0,2022-02-02 02:42:07,Edwards,Bill Sue,Edwards,suesellsrealestate@hotmail.com,64029,US,172.31.32.233,God Bless you and Keep you safe! Freedom for All!,100.0,10.0,0,0.0,0,NaN,NaN,NaN,ch_3KOYnJH7qsxOFoUf0END3Zaw,NaN,NaN,0,0,1,0,NaN,NaN,MasterCard,credit,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,464220,49000,NaN,22797,1,1,0,2022-02-02 02:47:11,MaryEllen Stevens,MaryEllen,Stevens,mest1115@aol.com,8822,US,172.31.32.233,Thank you! God bless you all and keep you safe. Godspeed you on your mission! â¤ï¸?â¤ï¸,25.0,3.0,0,0.0,0,NaN,NaN,NaN,ch_3KOYsDH7qsxOFoUf0KX1q0Sm,NaN,NaN,0,0,1,0,NaN,NaN,Visa,credit,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,464221,49000,NaN,22797,1,1,0,2022-02-02 02:47:11,Our turn.,Matthew,Klaasmeyer,adamklaasmeyer@gmail.com,85255,US,172.31.32.233,Glory to God.,10.0,1.0,0,0.0,0,NaN,NaN,NaN,ch_3KOYsEH7qsxOFoUf0QStinRy,NaN,NaN,0,0,1,0,NaN,NaN,MasterCard,prepaid,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,464223,49000,NaN,22797,1,1,0,2022-02-02 02:48:08,NaN,Steven,Lukomski,Stevenlukomski@yahoo.com,53027,US,172.31.32.233,Go Truckers!,20.0,2.0,0,0.0,0,NaN,NaN,NaN,ch_3KOYt9H7qsxOFoUf1wwCqvu7,NaN,NaN,0,0,1,0,NaN,NaN,MasterCard,credit,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [21]:
df = src[
    [
        "donation_id",
        "service_provider_credentials_id",
        "donation_date",
        "donation_name",
        "donation_first_name",
        "donation_last_name",
        "donation_zip",
        "donation_country",
        "donation_amount",
    ]
].copy()

In [25]:
us_df = df[df["donation_country"] == "US"].copy()

In [26]:
us_df["donation_zip"] = us_df["donation_zip"].str.zfill(5)

In [34]:
us_df[["donation_zip_clean", "drop", "drop"]] = us_df["donation_zip"].str.split(
    "-", expand=True
)

In [71]:
zips_df = (
    us_df.groupby(["donation_zip_clean"])
    .agg({"donation_amount": ["sum", "count"]})
    .reset_index()
)

In [76]:
zips_df.columns = zips_df.columns.to_series().str.join("_")

In [89]:
zips_df.rename(columns={"donation_zip_clean_": "zip"}, inplace=True)

In [93]:
zips_df.head()

In [162]:
zips_df["zip3"] = zips_df["zip"].str[:3]

In [189]:
zips_df.head()

,zip,donation_amount_sum,donation_amount_count,zip3
0,!3343,25.0,1,!33
1,&9770,10.0,1,&97
2,0000!,25.0,1,000
3,00000,190.0,3,000
4,00008,25.0,1,000


---

#### Aggregate by state

In [181]:
zips_lookup = pd.read_json("/Users/stiles/data/gis/zips_lookup_city_state.json")

In [187]:
zips_lookup["zip"] = zips_lookup["zip"].astype(str).str.zfill(5)

In [188]:
zips_lookup.head()

In [190]:
merge_zips = pd.merge(zips_df, zips_lookup, on="zip")

In [194]:
state_totals = (
    merge_zips.groupby(["statename"]).agg({"donation_amount_sum": sum}).reset_index()
)

In [200]:
states = pd.read_excel("/Users/stiles/data/states_population.xlsx")

In [203]:
states["state"] = states["state"].str.replace(".", "", regex=False)

In [226]:
states_merge = pd.merge(states, state_totals, left_on="state", right_on="statename")

In [227]:
states_merge["per100k"] = (
    states_merge["donation_amount_sum"] / states_merge["population"]
) * 100000

In [229]:
states_merge.sort_values("per100k", ascending=False).head()

,state,population,statename,donation_amount_sum,per100k
12,Idaho,1839106,Idaho,59268.0,3222.652745
1,Alaska,733391,Alaska,20838.0,2841.322023
47,Washington,7705281,Washington,177947.0,2309.416100
26,Montana,1084225,Montana,25005.0,2306.255620
34,North Dakota,779094,North Dakota,16625.0,2133.888850


In [230]:
states_merge.to_csv("data/processed/state_donations_percapita.csv", index=False)

---

## Geography

#### Read ZIP Codes

In [90]:
zips = gpd.read_file("/Users/stiles/data/gis/zipcodes.geojson")

In [91]:
zips.zip = zips.zip.astype(str)

In [134]:
merged = gpd.GeoDataFrame(pd.merge(zips_df, zips, on="zip"))

In [173]:
merged.sort_values("donation_amount_sum", ascending=False).head(20)

,zip,donation_amount_sum,donation_amount_count,city,state,geometry,x,y
11658,94062,95192.0,6,Redwood City,CA,POINT (-122.29000 37.40000),-122.29,37.40
8887,75024,20595.0,14,Plano,TX,POINT (-96.74000 33.04000),-96.74,33.04
10285,83814,19030.0,26,Coeur D Alene,ID,POINT (-116.91000 47.59000),-116.91,47.59
11174,91752,10300.0,8,Mira Loma,CA,POINT (-117.53000 33.99000),-117.53,33.99
9687,78701,10150.0,4,Austin,TX,POINT (-97.74000 30.26000),-97.74,30.26
5931,48844,10000.0,2,Howell,MI,POINT (-83.93000 42.60000),-83.93,42.60
11123,91362,9855.0,16,Thousand Oaks,CA,POINT (-118.81000 34.19000),-118.81,34.19
12470,98034,7895.0,19,Kirkland,WA,POINT (-122.20000 47.72000),-122.20,47.72
10943,90024,6710.0,11,Los Angeles,CA,POINT (-118.43000 34.06000),-118.43,34.06
11387,92592,6630.0,23,Temecula,CA,POINT (-117.04000 33.53000),-117.04,33.53


In [138]:
merged["x"] = merged.geometry.apply(lambda x: x.x).astype(float)
merged["y"] = merged.geometry.apply(lambda x: x.y).astype(float)

In [142]:
merged_slim = merged[
    ["zip", "donation_amount_sum", "donation_amount_count", "city", "state", "x", "y"]
]

In [167]:
zips_df.to_csv("data/processed/merged_zips_count_sum_three.csv", index=False)

In [136]:
merged.to_file("data/processed/merged_zips_count_sum.geojson", driver="GeoJSON")

In [178]:
merged.sort_values("donation_amount_sum", ascending=False).head(20).to_csv(
    "data/processed/merged_zips_count_sum.csv", index=False
)